## Collapse Board Data to Firm Level, Merge Preqin Returns Data

### For Table 5 of Elite Embeddedness

By Charlie Eaton and Albina Gibadullina

In [1]:
set line 150
use acceptrate alumni betweenness boardchair boardofficer control eigenvector financetype firmid firmname_new forbes forbesyear firstname middlename last suffix ///
hubs industry jd mba md phd schooltype strength undergrad unitid unique_id year yeargraduated schoolname using ///
"/Users/Charlie/Dropbox/Bankers in the Ivory Tower Book/2 Our New Financial Oligarchy/Data/Boards 2000 to 2017/board_members_all_long20220501.dta", clear
describe, size
save d_board_members_all_long20220501, replace





Contains data from /Users/Charlie/Dropbox/Bankers in the Ivory Tower Book/2 Our New Financial Oligarchy/Data/Boards 2000 to 2017/board_members_all_lon
> g20220501.dta
 Observations:        34,686                  
    Variables:            30                  02 May 2022 01:24
         Size:    29,483,100                  
------------------------------------------------------------------------------------------------------------------------------------------------------
Variable      Storage   Display    Value
    name         type    format    label      Variable label
------------------------------------------------------------------------------------------------------------------------------------------------------
unitid          double  %9.0g                 unitid
firstname       str16   %19s                  firstname
lastname        str61   %61s                  lastname
year            double  %9.0g                 
schoolname      str43   %43s                  schoolname

In [2]:
%head firstname lastname year firmid firmname_new eigenvector schoolname if firmname=="blackrock" & year==2017

,firstname,lastname,year,firmid,firmname_new,eigenvector,schoolname
3421,robert,kapito,2017,110,blackrock,.128855489306535,university of pennsylvania
5383,rick,rieder,2017,110,blackrock,.128855489306535,emory university
7020,barbara,novick,2017,110,blackrock,.128855489306535,cornell university
14509,laurence,fink,2017,110,blackrock,.128855489306535,new york university


In [3]:
use unitid firmid firmname_new eigenvector betweenness strength schooltype schoolname year using d_board_members_all_long20220501, clear
drop if schooltype=="liberal arts"
drop if unitid==232186 | unitid==126818 | unitid==209542 | unitid==221759 | unitid==102614 | unitid==129020 | unitid==181464

**correct delaware unitid (flagship which has enrollment data etc.)
replace unitid=130943 if unitid==130343
keep if year>2002 & year<2018



(1,886 observations deleted)

(1,689 observations deleted)

(574 real changes made)

(4,765 observations deleted)


In [4]:
merge m:1 unitid year using d_school_level_elite_embed, keepusing(alumnipehedge ///
     acceptrate fte_count endowment_assets total03_revenue schooltypenum ///
    ftfirstefwhitt ptfirstefwhitt ftfirstefasiat ptfirstefasiat ftfirsteftotlt ptfirsteftotlt ///
    pe hedge otherfinance nonfinance fgrnt_p year cpi_scalar_2017)

tab schooltypenum

qui egen repfirst=rowtotal(ftfirstefwhitt ptfirstefwhitt ftfirstefasiat ptfirstefasiat)
qui egen allfirst=rowtotal(ftfirsteftotlt  ptfirsteftotlt)
qui gen pctunderfirst= (allfirst-allfirst) / allfirst

gen privatetop30=schooltypenum==3
gen publictop30=schooltypenum==2
gen rejectrtindex=1-acceptrate
gen rejectrtmn=1-acceptrate
gen rejectmax=1-acceptrate



    Result                      Number of obs
    -----------------------------------------
    Not matched                             0
    Matched                            26,346  (_merge==3)
    -----------------------------------------


schooltypen |
         um |      Freq.     Percent        Cum.
------------+-----------------------------------
     public |      6,585       24.99       24.99
    private |     19,761       75.01      100.00
------------+-----------------------------------
      Total |     26,346      100.00










In [5]:
qui collapse (max) rejectmax (first) schoolname firmname_new eigenvector betweenness strength cpi_scalar_2017 (sum) fte_count endowment_assets total03_revenue ///
    privatetop30 publictop30 pe hedge otherfinance nonfinance alumnipehedge rejectrtindex (mean) pctunderfirst fgrnt_p acceptrate ///
    rejectrtmn, by(firmid year)

In [6]:
qui gen endow_assets_c_log=log(endowment_assets / cpi_scalar_2017)
qui gen logtotal03_revenue=log(total03_revenue / cpi_scalar_2017)
qui gen logfte_count=log(fte_count)

*qui egen pehedge=rowtotal(pe hedge)
*qui egen boardcount=rowtotal(pe hedge otherfinance nonfinance)
*qui gen pctalt= pehedge/boardcount

label var alumnipehedge "# financier alumni in Forbes 400"
label var logfte_count "log FTE enrollment"
label var pctunderfirst "% underrepresented students of color"
label var logtotal03_revenue "log total revenue"
label var endow_assets_c_log "log endowment value"
label var fgrnt_p "% Pell recipients"
label var acceptrate "Admissions rate"

In [7]:
save d_firm_level_trustee_ties, replace

file d_firm_level_trustee_ties.dta saved


In [8]:
import excel using Preqin_PEPerf_20210717195427.xlsx ///
, cellra(A22) case(lower) first clear 

. import excel using Preqin_PEPerf_20210717195427.xlsx , cellra(A22) case(lower
> ) first clear
(28 vars, 10,996 obs)


In [9]:
destring netirr benchnetirr netirrdiffpts, ignore("n" "/" "m" ",") replace


netirr: characters n / m removed; replaced as double
(2082 missing values generated)
benchnetirr: characters n / m removed; replaced as double
(970 missing values generated)
netirrdiffpts: characters n / m removed; replaced as double
(2113 missing values generated)


In [10]:
rename vintage year
drop if year<2003 | year>2017
tostring firmid, replace



(4,256 observations deleted)

firmid was long now str6


In [11]:
merge m:m firmid year using d_firm_level_trustee_ties, gen(trustmerge) keep(1 3)

. merge m:m firmid year using d_firm_level_trustee_ties, gen(trustmerge) keep(1
>  3)
(variable year was int, now double to accommodate using data's values)

    Result                      Number of obs
    -----------------------------------------
    Not matched                         6,278
        from master                     6,278  (trustmerge==1)
        from using                          0  (trustmerge==2)

    Matched                               462  (trustmerge==3)
    -----------------------------------------


In [12]:
destring firmid, replace
gen pctfundtarget= fundvaluemnusd / fundtargetvaluemnusd
replace fundvaluemnusd=fundvaluemnusd*1000000
replace fundtargetvaluemnusd=fundtargetvaluemnusd*1000000
gen lnfundvalue=ln(fundvaluemnusd)
gen lnfundtarget=ln(fundtargetvaluemnusd)


firmid: all characters numeric; replaced as long

(1,964 missing values generated)

(6,492 real changes made)

(4,878 real changes made)

(248 missing values generated)

(1,862 missing values generated)


In [13]:
foreach var in publictop30 privatetop30 eigenvector rejectrtindex rejectrtmn rejectmax {
    replace `var' = 0 if `var'==.
}
gen private=privatetop30!=0
gen public=publictop30!=0


(6,278 real changes made)
(6,278 real changes made)
(6,278 real changes made)
(6,278 real changes made)
(6,278 real changes made)
(6,278 real changes made)




In [14]:
save d_prequin_trustee_ties, replace

file d_prequin_trustee_ties.dta saved
